#Imports

In [ ]:
import os, sys, inspect, datetime

In [ ]:
# add the sccript path to opengrid to sys.path
script_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.join(script_dir, os.pardir, os.pardir))

from opengrid.library import config

##Get Forecast.io API key from config file

In [ ]:
c = config.Config()

#get Forecast.io API Key
api_key = c.get('Forecast.io', 'apikey')

##Import API wrapper module

In [ ]:
from opengrid.library import forecastwrapper

#Get weather data in daily resolution

In [ ]:
start = datetime.datetime(2015,8,13)
end = datetime.datetime(2015,8,16)

In [ ]:
WD = forecastwrapper.Weather_Days(api_key = api_key, location = 'Ukkel', start = start, end = end)

In [ ]:
WD.df

#Get weather data in hourly resolution

In [ ]:
WH = forecastwrapper.Weather_Hours(api_key = api_key, location = 'Ukkel', start = datetime.date(2015,8,23), end = datetime.date(2015,8,26))

In [ ]:
WH.df